In [7]:
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.regularizers import l2
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator
import datetime as dt
import dill
# Check current directory
import os
print(os.getcwd())

/Users/sethblum/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Courses/1C51/project-repo/FinalProject_1.C01


In [8]:
modelpath = 'Model_Outputs/model_BiLSTM_3layers_2024-04-29 12:25:08.646974.keras'
model = load_model(modelpath)


In [9]:
scalerX = dill.load(open('Xscaler.pkl', 'rb'))
scalerY = dill.load(open('Yscaler.pkl', 'rb'))
# Load the data
X_train_encoded = dill.load(open('X_train_encoded.pkl', 'rb'))
X_test_encoded = dill.load(open('X_test_encoded.pkl', 'rb'))
y_train_encoded = dill.load(open('y_train_encoded.pkl', 'rb'))
y_test_encoded = dill.load(open('y_test_encoded.pkl', 'rb'))
# TODO need to add
y_test_notscaled = dill.load(open('raw_y_test.pkl', 'rb'))

In [ ]:
# Define function to compare and plot model predictions from actual values
# Allow to zoom into a certain interval on the x axis

def plot_comparison(predictions, actual, startx=None, endx=None, starty=None, endy=None):
    """
    Plot the predicted vs actual values with an optional x-axis limit.

    Parameters:
        predictions (array-like): The predicted values.
        actual (array-like): The actual values.
        start (int, optional): Start index for x-axis limit.
        end (int, optional): End index for x-axis limit.
    """
    plt.figure(figsize=(12, 6))
    plt.plot(predictions, label='Predicted', marker='o')  # Added markers for better visibility
    plt.plot(actual, label='Actual', marker='x')  # Added markers for better visibility
    plt.legend()
    
    # Set x-axis limits if specified
    if startx is not None or endx is not None:
        plt.xlim(startx, endx)
        
    # Set y-axis limits if specified
    if starty is not None or endy is not None:
        plt.ylim(starty, endy)    
    
    plt.title('Comparison of Predictions and Actual Values')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.show()

    
    # Plot the differences between the actual value and the predicted value
    plt.figure(figsize=(12, 6))
    
    # Set x-axis limits if specified
    if startx is not None or endx is not None:
        plt.xlim(startx, endx)
        
    # Set y-axis limits if specified
    if starty is not None or endy is not None:
        plt.ylim(starty, endy)    
    
    plt.plot(predictions - actual, label='Difference')
    plt.legend()
    print('everything but show')
    plt.show()
    print('done')
    return
    

In [10]:
predictions = model.predict(X_test_encoded)
predictions = scalerY.inverse_transform(predictions)
predictions = predictions.reshape(-1)


865/865 [==============================] - 14s 15ms/step


In [ ]:
# TODO - realign the predictionns with the true RTM and DAM prices and dates

pred_df = pd.DataFrame({'Actual': y_test_notscaled, 'Predicted': predictions})

shifted_predictions = np.roll(predictions, 288)
plot_comparison(shifted_predictions, y_test_nonscale, startx=0, endx=0+96*8, starty=0, endy=100)